# - Playing LDA on the arxiv -
If you have sklls on this and/or are interested in contributing, you are welcome!

feedback: sequi@unsam.edu.ar Ezequiel Alvarez

## Set up 0: choose hep-th, hep-ph ot cond-mat 
uncomment the one you want

In [7]:
field="hep-ph"
#field="hep-th"
#field="cond-mat"

## Set up

In [8]:
import pandas as pd
import pickle
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
np.random.seed(2018)
from random import randint
from operator import add
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Load Saved Model

In [9]:
lda_model = gensim.models.LdaMulticore.load("models/"+field)

In [10]:
data = pd.read_pickle('data/data_'+field+'.pickle');
with open ('data/bow_corpus_'+field+'.pickle','rb') as file:
    bow_corpus = pickle.load(file)

In [11]:
Ntopics=0;
for idx, topic in lda_model.print_topics(-1):
    Ntopics=Ntopics+1;
print(Ntopics)

15


## Learn your preferences 

In [16]:
print("We will ask you to rank a few papers between -10 (worst) and +10 (best).  Try to be accurate, since from these answers the machine will learn your preferences for the next steps!\n\n")
input("Press [enter] to start!")
topicscore = [0] * Ntopics
for i in range(20):
    docnumber = randint(0,len(bow_corpus))
    print("\n"+color.BOLD+data.iloc[docnumber,2]+color.END+"\n\n"+data.iloc[docnumber,3])
    score = int(input("Score this paper between -10 (worst) and +10 (best): "))
    score2add = [row[1]*score for row in lda_model[bow_corpus[docnumber]]]
    if ( len(score2add) != Ntopics ):
        print('Not good length')
    if ( len(score2add) == Ntopics ):
        topicscore=list( map(add, topicscore, score2add) )
        print("----")
     
#print("\n\nGood, your score over topics is:\n")
#print(topicscore)
topicscore = np.array(topicscore)

We will ask you to rank a few papers between -10 (worst) and +10 (best).  Try to be accurate, since from these answers the machine will learn your preferences for the next steps!


Press [enter] to start!

Quantum String Dynamics in the conformal invariant SL(2,R) WZWN
  Background: Anti-de Sitter Space with Torsion

We consider classical and quantum strings in the conformally invariant
background corresponding to the SL(2,R) WZWN model. This background is locally
anti-de Sitter spacetime with non-vanishing torsion. Conformal invariance is
expressed as the torsion being parallelized. The precise effect of the
conformal invariance on the dynamics of both circular and generic classical
strings is extracted. In particular, the conformal invariance gives rise to a
repulsive interaction of the string with the background which precisely cancels
the dominant attractive term arising from gravity. We perform both
semi-classical and canonical string-quantization, in order to see the effect of
th

----

Associated production of $\chi_c$ pairs with a gluon in the
  collinear-factorization approach

We calculate cross section for production of $\chi_c$ pairs in proton-proton
collisions. The cross section for the $g g \to \chi_{c J_1} \chi_{c J_2}$ is
considerably smaller (especially for $\chi_{c 1} \chi_{c 1}$ final state) than
that obtained recently in the $k_T$-factorization approach. We calculate
therefore next-to-leading order contributions with $\chi_c$ pair and one extra
associated (mini-)jet. We find these contributions to be much larger than those
for the $2 \to 2$ contribution. Especially the emission of a leading gluon
(carrying a large momentum fraction of one of the incoming gluons) are
important. These emissions in the $k_T$-factorization approach are absorbed
into the initial state unintegrated gluon distributions. A smaller contribution
to the cross section comes from the production of central gluons emitted with
rapidities between the $\chi_c$-mesons. They do lead,

Score this paper between -10 (worst) and +10 (best): -1
----

Kramers's escape rate problem within a non-Markovian description

We compare the thermal escape rates of a Brownian particle, initially trapped
into one of the two wells of an asymmetric double-well potential, for thermal
Markovian and non-Markovian noise. The Markovian treatment of this problem goes
originally back to the studies of Kramers in 1940 and is therefore often
referred to as "Kramers's escape rate problem". We solve the generalized
Langevin equation for the trajectories of the particles numerically and
analytically for both limiting cases, Markovian and non-Markovian thermal
noise. We compute the escape rate and work out the fundamental differences
arising from finite correlation times of the thermal noise.
Score this paper between -10 (worst) and +10 (best): -2
----

In the Shadow of the Color Glass

I give a brief overview of recent theoretical progress within perturbative
QCD concerning the high-energy dynamic

### Game 1: Sort 10 random papers according to your preferences

In [18]:
out = pd.DataFrame(columns=['score', 'title'])
for i in range(10):
    docnumber = randint(0,len(bow_corpus))
    thistitle=np.array([row[1] for row in lda_model[bow_corpus[docnumber]]])
    if ( len(thistitle) == Ntopics ):
        thisscore=thistitle.dot(topicscore)
    out = out.append({'score': thisscore, 'title':data.iloc[docnumber,2]},ignore_index=True)
pd.options.display.max_colwidth = 150
display(out[['title']])
input("Press [enter] to see how I guess your preferences!")
display(out.sort_values('score',ascending=False))


,title
0,Confirming the Existence of the strong CP Problem in Lattice QCD with\n the Gradient Flow
1,Towards the Finite Temperature Gluon Propagator in Landau Gauge\n Yang-Mills Theory
2,"Chiral-Odd Generalized Parton Distributions, Transversity and Double\n Transverse-Spin Asymmetry in Drell-Yan Dilepton Production"
3,The Higgs Profile in the Standard Model and Beyond
4,Absorption of Fermionic Dark Matter by Nuclear Targets
5,Aspects of quantum gravity
6,Phase Transitions in the Core of Global Embedded Defects
7,HE Stratosphere Event of 1975 Revisited: New Physics in Astroparticle\n Collision vs. LHC Nucleus-Nucleus Data
8,Do We Understand the Sphaleron Rate?
9,Initial conditions for electron and photon structure and fragmentation\n functions


Press [enter] to see how I guess your preferences!


,score,title
8,2.780490,Do We Understand the Sphaleron Rate?
4,2.544386,Absorption of Fermionic Dark Matter by Nuclear Targets
6,2.522267,Phase Transitions in the Core of Global Embedded Defects
7,2.474859,HE Stratosphere Event of 1975 Revisited: New Physics in Astroparticle\n Collision vs. LHC Nucleus-Nucleus Data
3,2.261074,The Higgs Profile in the Standard Model and Beyond
0,1.732343,Confirming the Existence of the strong CP Problem in Lattice QCD with\n the Gradient Flow
9,1.660640,Initial conditions for electron and photon structure and fragmentation\n functions
2,1.540604,"Chiral-Odd Generalized Parton Distributions, Transversity and Double\n Transverse-Spin Asymmetry in Drell-Yan Dilepton Production"
5,1.185934,Aspects of quantum gravity
1,0.437768,Towards the Finite Temperature Gluon Propagator in Landau Gauge\n Yang-Mills Theory


# Game 2: Match between 2 papers

In [ ]:
    for hh in range(10):
     docnumber1 = randint(0,len(bow_corpus))
     docnumber2 = randint(0,len(bow_corpus))     
     thistitle1=np.array([row[1] for row in lda_model[bow_corpus[docnumber1]]])
     thistitle2=np.array([row[1] for row in lda_model[bow_corpus[docnumber2]]])
     print("Paper A: "+color.BOLD+data.iloc[docnumber1,2]+color.END+'\n'+data.iloc[docnumber1,3]+'\n')
     print("Paper B: "+color.BOLD+data.iloc[docnumber2,2]+color.END+'\n'+data.iloc[docnumber2,3]+'\n')
     input("Press [enter] to see how I guess your preferences!")
     print ("\nThis is my preference:\n")
     score1=thistitle1.dot(topicscore)
     score2=thistitle2.dot(topicscore)
     if (score1 > score2):
            print("1)"+data.iloc[docnumber1,2]+"\t (Score: "+str(round(score1,2))+")")
            print("2)"+data.iloc[docnumber2,2]+"\t (Score: "+str(round(score2,2))+")"+"\n--------------\n")
     else:
            print("1)"+data.iloc[docnumber2,2]+"\t (Score: "+str(round(score2,2))+")")
            print("2)"+data.iloc[docnumber1,2]+"\t (Score: "+str(round(score1,2))+")"+"\n--------------\n")
     input("Press [enter] to see next papers")

# Bonus Track: get your most interesting papers in the Corpus!

In [ ]:
allscores=[]
for docnumber in range(len(data)):
    thistitle=np.array([row[1] for row in lda_model[bow_corpus[docnumber]]])
    thisscore=thistitle.dot(topicscore)
    allscores.append(thisscore)
data['score'] = allscores
sorted_data=data.sort_values('score',ascending=False)

In [ ]:
display(sorted_data[['title','authors','score']][:20])